In [215]:
################ Library imports ################
import numpy as np
import math as math

from matplotlib import pyplot as plt

from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel, WhiteKernel

from sklearn.metrics import mean_squared_error, mean_absolute_error

In [216]:
# ----------------------------------------------------------------------
#  AP locations
L_ap = np.array([[0, 0], [0, 2], [2, 0], [2, 2]])

# ----------------------------------------------------------------------
#  VRP locations
L_vrp = np.array([[1, 0], [0, 1], [1, 1], [1, 2], [2, 1]])

# ----------------------------------------------------------------------
#  RTT distance observations
d_RTT = np.array([0, 2, 2, math.sqrt(8)])

In [217]:
"""
Gaussian Process Kernel definition

Kernel parameters:
    - L --> Length Scale
    - sigma_f --> Constant Kernel Amplitude
"""

l = 0.1
sigma_f = 2

kernel = ConstantKernel(constant_value=sigma_f,constant_value_bounds=(1e-3, 1e3)) \
            * RBF(length_scale=l, length_scale_bounds=(1e-3, 1e3))  

In [218]:
"""
Gaussian Process Regressor definition

- sigma_n --> Training Noise Variance
"""

# Error standard deviation. 
sigma_n = 0.2

gp = GaussianProcessRegressor(kernel=kernel, alpha=sigma_n**2, n_restarts_optimizer=10)

In [228]:
"""
Model Fitting with AP location and RTT distance observations
using Maximum Likelihood Estimation of the parameters.
"""

gp.fit(L_ap, d_RTT)

GaussianProcessRegressor(alpha=0.04000000000000001,
                         kernel=1.41**2 * RBF(length_scale=0.1),
                         n_restarts_optimizer=10)

In [229]:
"""
Prediction of the VRPs around the scenario
"""
d_RTT_pred, d_RTT_err = gp.predict(L_vrp, return_std=True)

print(d_RTT_pred)
print(d_RTT_err)

[1.08011443 1.08011443 1.9508464  2.56356696 2.56356696]
[0.2747788  0.2747788  0.34364533 0.2747788  0.2747788 ]


In [230]:
"""
Optimized parameters of the Kernel
"""
gp.kernel_.get_params()

{'k1': 2**2,
 'k2': RBF(length_scale=2.47),
 'k1__constant_value': 4.001437979296082,
 'k1__constant_value_bounds': (0.001, 1000.0),
 'k2__length_scale': 2.4737020613973186,
 'k2__length_scale_bounds': (0.001, 1000.0)}

In [231]:
"""
Log Marginal Likelihood of the obtained parameters
"""
gp.log_marginal_likelihood()

-7.062481224146938

In [232]:
"""
Mean squared error and variance of the prediction
"""
d_RTT_true = np.array([1, 1, math.sqrt(2), math.sqrt(5), math.sqrt(5)])
print("MSE: ", mean_squared_error(d_RTT_true, d_RTT_pred))
print("ME: " , mean_absolute_error(d_RTT_true, d_RTT_pred))
print("STD: " , np.std(d_RTT_true-d_RTT_pred))

MSE:  0.10306452132806518
ME:  0.2703719317996808
STD:  0.1730997972932781
